In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# use CMU Serif
import matplotlib as mpl
import matplotlib.font_manager as font_manager
mpl.rcParams['font.family']='serif'
# cmfont = font_manager.FontProperties(fname=mpl.get_data_path() + '/fonts/ttf/cmr10.ttf')
cmfont = font_manager.FontProperties('/Users/theoares/Library/Fonts/cmunrm.otf')
mpl.rcParams['font.serif']=cmfont.get_name()
mpl.rcParams['mathtext.fontset']='cm'
mpl.rcParams['axes.unicode_minus']=False

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib.transforms import Bbox

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools

n_boot = 50
from utils import *

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['/Users/theoares/Library/Fonts/cmunrm.otf'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


## Play with the Gqq, Gqg correlators

In [2]:
l = 48
t = 96
L = Lattice(l, t)

In [3]:
# read in qlua output
jobid = 90712
data_dir = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_' + str(jobid)
unsorted_cfgs = []
unsorted_cfgids = []
for (dirpath, dirnames, file) in os.walk(data_dir):
    unsorted_cfgs.extend(file)
    for fi in file:
        unsorted_cfgids.append(int(fi[3 : 7]))    # slicing only good for cfgs in the 1000's
for idx, cfg in enumerate(unsorted_cfgs):
    unsorted_cfgs[idx] = data_dir + '/' + unsorted_cfgs[idx]
n_cfgs = len(unsorted_cfgs)
print('Reading ' + str(n_cfgs) + ' configs.')
print(unsorted_cfgids)

Reading 32 configs.
[1480, 1490, 1390, 1380, 1170, 1160, 1280, 1140, 1290, 1300, 1240, 1180, 1310, 1270, 1330, 1320, 1260, 1220, 1360, 1370, 1230, 1350, 1210, 1200, 1340, 1470, 1460, 1500, 1450, 1400, 1420, 1430]


In [4]:
# get glue pieces
glue_dir = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/glue_ops/cfgs_1100_1500/'
path_glue_ids = glue_dir + 'ops_for_patrick.txt'
path_glue_emt = glue_dir + 'ops_for_patrick.npy'
glue_emt_all = np.load(path_glue_emt)
glue_ids = []
with open(path_glue_ids, 'r') as f:
    for line in f:
        glue_ids.append(int(line[line.find('cfg_') + len('cfg_') : -1]))

In [5]:
# sort all cfgs
cfgids = sorted(unsorted_cfgids)
cfgs = [x for _, x in sorted(zip(unsorted_cfgids, unsorted_cfgs), key = lambda pair: pair[0])]
glue_emt = np.array([glue_emt_all[ii] for ii in range(len(glue_ids)) if glue_ids[ii] in cfgids])

In [33]:
glue_emt.shape

(32, 4, 4)

In [34]:
# read in quark / glue stuff and bootstrap
k = np.array([3, 0, 0, 0])
props, Gqg_qlua, Gqq3, Gqq6 = readfiles(cfgs, k)
props_b = bootstrap(props)
glue_emt_b = bootstrap(glue_emt)
Gqq3_b = np.array([bootstrap(Gqq3[a]) for a in range(3)])
Gqq6_b = np.array([bootstrap(Gqq6[a]) for a in range(6)])
Gqg = np.einsum('zmn,zaibj->mnzaibj', glue_emt, props)
Gqg3, Gqg6 = form_2d_sym_irreps(Gqg)
Gqg3_b = np.array([bootstrap(Gqg3[a]) for a in range(3)])
Gqg6_b = np.array([bootstrap(Gqg6[a]) for a in range(6)])

In [35]:
np.mean(glue_emt_b, axis = 0)

array([[-6.3871681e+05+0.j,  9.6702148e+01+0.j, -1.9694864e+00+0.j,
         1.1603106e+02+0.j],
       [ 9.6702148e+01+0.j, -6.3890925e+05+0.j,  1.6075146e+02+0.j,
         1.0425608e+02+0.j],
       [-1.9694864e+00+0.j,  1.6075146e+02+0.j, -6.3917538e+05+0.j,
         6.9358940e+01+0.j],
       [ 1.1603106e+02+0.j,  1.0425608e+02+0.j,  6.9358940e+01+0.j,
        -6.3893681e+05+0.j]], dtype=complex64)

In [36]:
np.std(glue_emt_b, axis = 0, ddof = 1)

array([[453.12097 ,  94.33288 ,  90.05134 , 112.573074],
       [ 94.33288 , 432.63602 , 140.45341 , 101.8259  ],
       [ 90.05134 , 140.45341 , 400.75943 , 150.34732 ],
       [112.573074, 101.8259  , 150.34732 , 473.7684  ]], dtype=float32)

In [38]:
props_b.shape

(50, 3, 4, 3, 4)

In [39]:
props_D = np.einsum('zaiaj->zij', props_b) / 3.

In [40]:
np.mean(props_D, axis = 0)

array([[ 0.6001193 +0.01388297j,  0.11062092-0.03946992j,
        -0.0354077 +0.26665348j, -2.266762  +0.06819782j],
       [-0.17416838+0.0016999j ,  0.4760185 -0.07568324j,
        -2.2908564 -0.07031874j,  0.00302245+0.14962615j],
       [-0.04041865+0.10283693j,  2.123166  +0.10173002j,
         0.6768617 -0.03703861j, -0.10520068-0.02382746j],
       [ 2.3535724 +0.19488555j,  0.10865663+0.24889278j,
         0.05968416+0.0539181j ,  0.57171124-0.12851085j]],
      dtype=complex64)

In [41]:
np.std(props_D, axis = 0, ddof = 1)

array([[0.11369263, 0.12933907, 0.17663124, 0.19157533],
       [0.18419436, 0.12328433, 0.22127399, 0.2108518 ],
       [0.18269148, 0.16041623, 0.09999421, 0.10924498],
       [0.15745665, 0.21943788, 0.12539428, 0.13177463]], dtype=float32)

In [27]:
Gqq3_mu = np.mean(Gqq3_b, axis = 1)
Gqq3_sigma = np.std(Gqq3_b, axis = 1, ddof = 1)
Gqq6_mu = np.mean(Gqq6_b, axis = 1)
Gqq6_sigma = np.std(Gqq6_b, axis = 1, ddof = 1)
Gqg3_mu = np.mean(Gqg3_b, axis = 1)
Gqg3_sigma = np.std(Gqg3_b, axis = 1, ddof = 1)
Gqg6_mu = np.mean(Gqg6_b, axis = 1)
Gqg6_sigma = np.std(Gqg6_b, axis = 1, ddof = 1)

In [25]:
Gqq3_mu.shape

(3, 3, 4, 3, 4)

In [30]:
np.einsum('zaiaj->zij', Gqg3_mu) / 3

array([[[-109.898346 +21.570654j,  104.25478 +112.323j   ,
          138.91371  -35.23636j ,  474.62875   -7.816137j],
        [  36.080444-143.74368j , -176.81961  +97.73282j ,
          377.87463  +13.434141j,   24.189758-205.0589j  ],
        [  35.387196 -65.01332j , -195.83664  -38.614273j,
         -205.76334  +17.668821j,   38.030087 -14.852308j],
        [ -70.20056  +74.20444j , -215.43706 +134.58197j ,
          -83.766785 -94.156204j, -124.03762 +111.497986j]],

       [[ 106.86258 +205.8435j  ,   50.94621  -42.34877j ,
          -72.38201 -170.26756j , -563.7741    -8.01104j ],
        [-209.931   -176.44467j ,   10.376883 -72.62372j ,
         -335.57526  +46.346245j, -107.07656  +42.352383j],
        [-234.16856   +7.114698j,  186.23203  -90.2619j  ,
          -77.06147  +12.300548j,  -18.166006 -17.538542j],
        [ 365.66638  +76.08201j , -151.63387  +32.26178j ,
           21.6291   +63.981842j,   55.959152-225.31468j ]],

       [[ 291.95392  -11.886917j,  175.02808

In [31]:
np.einsum('zaiaj->zij', Gqg3_sigma) / 3

array([[[196.84344, 166.82385, 328.72766, 429.60147],
        [195.12384, 201.78094, 400.27417, 332.77615],
        [244.34778, 416.14343, 203.12885, 144.53957],
        [371.71762, 334.68808, 200.89616, 252.53978]],

       [[263.38254, 211.27281, 314.9628 , 418.91818],
        [291.4181 , 190.01436, 385.0027 , 255.26024],
        [297.96924, 460.98807, 215.04732, 163.82426],
        [401.56265, 327.37378, 147.30672, 192.15459]],

       [[278.89505, 261.34207, 318.9751 , 475.91678],
        [312.72464, 279.17642, 442.20877, 325.13876],
        [334.59485, 492.26535, 207.98125, 178.1148 ],
        [491.6443 , 335.16074, 176.06221, 214.61772]]], dtype=float32)

## See what the stats look like

In [ ]:
# try running on 4 nodes instead of 5-- not sure why the second job (89346) has bad data

In [ ]:
l = 48
t = 96
# L = Lattice(l, t)

In [ ]:
# path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/analysis_output/Z_89127.h5'
# f = h5py.File(path, 'r')
# k_list = f['momenta'][()]
# mom_list = [L.to_linear_momentum(k + bvec, datatype = np.float64) for k in k_list]
# apsq_list = [square(p) for p in mom_list]
# Zq = f['Zq'][()]
# Gamma_qg = f['Gamma_qg'][()]
# Gamma_qq3 = f['Gamma_qq3'][()]
# Gamma_qq6 = f['Gamma_qq6'][()]
# f.close()

In [ ]:
# colors = ['b', 'b', 'b', 'b', 'r', 'r', 'r']

In [ ]:
# Zq_mu = np.mean(Zq, axis = 1)
# Zq_std = np.std(Zq, axis = 1, ddof = 1)
# plt.figure()
# plt.errorbar(apsq_list[:4], Zq_mu[:4], Zq_std[:4], c = 'r', fmt = '.', label = 'h(p) = 0.25')
# plt.errorbar(apsq_list[4:], Zq_mu[4:], Zq_std[4:], c = 'b', fmt = '.', label = 'h(p) = 1.0')

# plt.xlabel('$(ap)^2$')
# plt.ylabel('$Z_q$')
# plt.title('Quark renormalization on 44 configurations')
# plt.legend()
# plt.show()

In [ ]:
# Gamma_qq3_red = np.abs(np.einsum('...aiaj->...ij', Gamma_qq3))
# Gamma_qq_red_mu = np.mean(Gamma_qq3_red, axis = 2)
# Gamma_qq_red_std = np.std(Gamma_qq3_red, axis = 2, ddof = 1)

In [ ]:
# plt.figure()
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 0], Gamma_qq_red_std[:, 0, 0, 0], c = 'b', fmt = '.', label = '11')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 1], Gamma_qq_red_std[:, 0, 0, 1], c = 'r', fmt = '.', label = '12')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 2], Gamma_qq_red_std[:, 0, 0, 2], c = 'g', fmt = '.', label = '13')
# plt.errorbar(apsq_list, Gamma_qq_red_mu[:, 0, 0, 3], Gamma_qq_red_std[:, 0, 0, 3], c = 'k', fmt = '.', label = '14')

# plt.xlabel('$(ap)^2$')
# plt.ylabel('$Γ1_{1i}$')
# plt.title('Components of vertex function for $\mathcal{O}_{33} - \mathcal{O}_{44}$ on 15 configurations')
# plt.legend()
# plt.show()

## Playing with some actual data

In [ ]:
# cfgs which had gf problems: 1020, 1100, 1190
# cfgs which had issues writing measurement to file: 1110, 1140, 1180

In [ ]:
# for cfg_idx in range(1120, 1500, 10):
#     f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg' + str(cfg_idx) + '.h5'
#     try:
#         f2 = h5py.File(f2path, 'r')
#         print('good file at ' + str(cfg_idx))
#         print(np.einsum('iiaa->', f2['Gqq/31/p3000'][()]))
#         f2.close()
#     except:
#         print('broken file at ' + str(cfg_idx))

In [ ]:
# for cfg_idx in range(1120, 1500, 10):
#     f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg' + str(cfg_idx) + '.h5'
#     try:
#         f2 = h5py.File(f2path, 'r')
#         print('good file at ' + str(cfg_idx))
#         print(f2['Gqq'])
#         f2.close()
#     except:
#         print('broken file at ' + str(cfg_idx))

In [ ]:
# # f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_89157/cfg1110.h5'
# # f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_90712/cfg1240.h5'
# f2path = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cfg1140.h5'
# f2 = h5py.File(f2path, 'r')
# print(f2['Gqq/31/p6000'][()])
# f2.close()

In [ ]:
# # some of the files are corrupted, which you can check in the slurm output (although they still wrote to the file). 
# # however, some of them are working fine. Going to try to run on the bad files with more nodes and see what happens
# # for the corrupted files, see what the correlator orders of magnitude are-- currently one Gqq for 1140 (one of the 
# # ones which errored) is having some major issues
# fpath = '/Users/theoares/Dropbox (MIT)/research/gq_mixing/meas/cl21_48_96_b6p3_m0p2416_m0p2050_89127/cfg1050.h5'
# f = h5py.File(fpath, 'r')
# print(f['Gqq/31/p6000'][()])

In [ ]:
# f['prop/p2222'][()]